In [ ]:
# Works best with jupyter-notebook

In [ ]:
%matplotlib notebook 
#%matplotlib widget 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
import numpy as np

from roboticstoolbox import *
import roboticstoolbox.tools.trajectory as tr

import matplotlib.pyplot as plt
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# Lecture 4.4 Multidimensional Smooth Trajectories

In this notebook, we set out to learn the implications of working with multile dimensions. Namely, a robot with more than one motor/degree of freedom; or alternatively, working on the xy-plane for example.

In this section, we will leverage the function **jtraj** to learn to create smooth multi-dimensional motions with initial and final positions under a desired number of timesteps. 

Begin by setting your start and end joint positions:

In [ ]:
#Set a first point of (10, 20) and a last point of (30, 10):
first = np.array([10,20])
last  = np.array([30,10])

print(first, last)

Compute a trajectory using the toolbox function jtraj(), 
accepting arguments of the first point, the last point and the number of timesteps:

tg = jtraj(q0, qf, M): is a joint space trajectory where the joint
coordinates vary from ``q0`` (N) to ``qf`` (N).  

A quintic (5th order) polynomial is used with default zero boundary conditions for velocity and acceleration.

The method returns a named tuple just as with tpoly and mstraj, but in this case we have a difference for time where we use t instead of x:
- t: time
- q: position
- dq: velocity
- qdd: acceleration

In [ ]:
timesteps = 10
out = tr.jtraj(first, last, timesteps)
print(out)

Time:

In [ ]:
print(out.t) # pos, vel, acc

Positions:

In [ ]:
print(out.q)   # trajectory

Velocities:

In [ ]:
print(out.qd)

Accelerations:

In [ ]:
print(out.qdd)

Let us plot the trajectory to facilitate visualization

In [ ]:
fig1 = plt.figure(1);

plt.xlabel('Time (secs)');
plt.ylabel('Position (x)');
plt.plot(out.q);

Plot the velocity graph for the two joints over time

In [ ]:
fig2 = plt.figure(2)

plt.xlabel('Time (secs)');
plt.ylabel('velocity xd (units/sec)');
plt.plot(out.qd);

You may also desire to specify initial and final velocities to each axis. 

In this case we will ask for the initial velocity to be 0 units/second and the final velocity to be 10 units/second in each axis:

In [ ]:
first = np.array([10,20])
last  = np.array([30,10])

qd0   = np.array([0,0])
qdf   = np.array([10,10])

out = tr.jtraj(first, last, timesteps, qd0, qdf)

In [ ]:
t   = out.t     # time coordinate
x   = out.q     # trajectory
xd  = out.qd    # velocity
xdd = out.qdd   # acceleration

Let's plot the velocity once again to compare the profiles with different final conditions

In [ ]:
#plot velocity
fig3=plt.figure(3)
plt.xlabel('time (secs)');
plt.ylabel('pos x (units/sec)');
plt.plot(xd);

## 2D Example with Via-Points

A 2D trajectory with via points can also be computed. But to do this, we will need indendent vectors with blends. 

We can use mstraj as before. 

Set a first point of (40, 50) and via points at (60, 30), (40, 10), (20, 30) and returning back to the start position. It makes a diamond, starting at the top vertex and then moving clock-wise.

In [ ]:
start=np.array([40,50])
via=np.array([
    [40,50],
    [60,30],
    [40,10],
    [20,30],
    [40,50]
    ])
print("Starting point at top of the diamond is: ", start)
print("Then move clock-wise around the diamond: \n", via)

---
Compute the trajectory using the toolbox function **mstraj()**.


Let us frist use simple numbers, later you can modify variable values. 

Provide:
- **multi-dimensional via points**

- a velocity of 1 units/second, 
- null segment velocities, 
- the start position, 
- the time step of 1 seconds 
- the acceleration duration time of 0 seconds:

In [ ]:
dt   = 1
tacc = 0
qdmax= 1

out = tr.mstraj(via, dt, tacc, qdmax, q0=start)

Time duration:

In [ ]:
print("The trajectory lasts a total of ", out.t[-1]+dt, " secs.")

Arrival times:

In [ ]:
print("The segments arrival times are as follows \n", out.arrive)

Position information:

In [ ]:
print("The actual trajectory is: ", out.q)

Via point information:

In [ ]:
print("My via points are: \n", out.via.squeeze())

Plot the figure of the joint angles for each joint. 

In [ ]:
# Plot
fig4=plt.figure(4)
plt.plot(out.q);

There is another interesting way to plot this graph. 

Plotting the first coordinate versus the second coordinate shows the motion of the point on the xy-plane:

In [ ]:
fig5=plt.figure(5)
plt.plot(out.q[:,0], out.q[:,1], color='black');

plt.xlabel('x');
plt.ylabel('y');

## What if we want each axis to move at a different velocity?

The velocity of each axis can be specified separately by passing a velocity vector qdmax. 

In this case a velocity of 1 unit/second in the x-axis and 3 units/second in the y-axis:

In [ ]:
dt   = 0.1
tacc = 1.0
qdmax = [1,3]

out=tr.mstraj(via, dt, tacc, qdmax, q0=start)

We can plot independent velocity profiles

In [ ]:
fig6 = plt.figure(6)
plt.plot(out.q);

What do you notice about the duration of the above graph? 

What is the ratio of qdmax before to the slower axis now?

---
We can again plot the 1st vs 2nd coordinate to show the xy motion:


In [ ]:
fig7 = plt.figure(7)

plt.plot(out.q[:,0], out.q[:,1]);
plt.xlabel('x');
plt.ylabel('y');

mstraj still generates smooth motions for both dimensions. It does so by first identifying the slower joint and using that information to compute the trajectory for the other joints.